In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import subprocess

In [2]:
%cd /Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models


In [3]:
train_results_dir = os.getcwd() + "/../002-Training-models/train_results/"
# !ls {train_results_dir}

In [4]:
# !mkdir trained_on_v2
%cd trained_on_v2
# !mkdir logs

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v2


In [5]:
eval_results_dir = os.getcwd()
dataset_dir = os.getcwd()+"/../../datasets/"


In [11]:
output_file_name = "eval_trained_on_dataset_v2"
output_file = f"logs/{output_file_name}.txt"

In [6]:
# Download YOLOv8
!pip install ultralytics

from IPython import display
display.clear_output()

%cd {eval_results_dir}
!mkdir yolov8

from ultralytics import YOLO
print('YOLOv8 setup complete.')

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v2
YOLOv8 setup complete.


In [9]:
%cd {eval_results_dir}

/Users/florenciavela/Desktop/flor/git/gun-detection/003-Evaluating-models/trained_on_v2


# Evaluate vs original datasets

In [ ]:
total = len(os.listdir(train_results_dir))
counter = 0
for folder_name in sorted(os.listdir(train_results_dir)):
    counter += 1
    if not folder_name.startswith("model"):
        continue

    folder_path = os.path.join(train_results_dir, folder_name, "weights")
    print(f"{counter}/{total}")

    # Split the folder name into variables
    variables = folder_name.split('_')

    # Extract the required variables
    model = variables[1]
    tf = variables[3]
    imgsz = variables[5]
    epochs = variables[7]
    batch = variables[9]
    dataset = variables[11]
    loss = variables[13]
    lr = variables[15]
    
    print(folder_name)
    print("Model:", model)
    print("TF:", tf)
    print("Imgsz:", imgsz)
    print("Epochs:", epochs)
    print("Batch:", batch)
    print("Dataset:", dataset)
    print("Executing command...")
    print()
    
    dataset_location = dataset_dir + model + "_original_v2"
        
    command = None
    
    if model == "yolov5" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov5
            command = f"python3 val.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov7" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov7
            command = f"python3 test.py --weights {best_pt_path} --data {dataset_location}/data.yaml --task test --name {folder_name}"
            print(command)

    elif model == "yolov8" and dataset == "v2":
        best_pt_path = os.path.join(folder_path, "best.pt")
        if os.path.exists(best_pt_path):
            %cd yolov8
            command = f"yolo val model={best_pt_path} data={dataset_location}/data.yaml split=test name={folder_name}"
            print(command)

    if command:
        # Run the command using subprocess.Popen and capture the output
        with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True) as process:
            # Open the output file for writing
            with open(f"{eval_results_dir}/{output_file}", "a") as file:
                file.write(folder_name)
                file.write(f"\nModel: {model}")
                file.write(f"\nTF: {tf}")
                file.write(f"\nImgsz: {imgsz}")
                file.write(f"\nEpochs: {epochs}")
                file.write(f"\nBatch: {batch}")
                file.write(f"\nDataset: {dataset}")
                file.write(f"\nLoss: {loss}")
                file.write(f"\nLr0: {lr}\n")
                # Read and display each line of the output as it becomes available
                for line in process.stdout:
                    print(line, end="")
                    file.write(line)
        %cd ..

    from IPython import display
    display.clear_output()
    

print("Executing commands...", "DONE!")
print()

## Convert the logs to CSV

In [10]:
!mkdir {eval_results_dir}/output_csvs

In [12]:
def process_file(file_path):
    results = list()
    curr_res = dict()

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        if line.startswith("Model:"):
            curr_res["model"] = line.split()[1]
        elif line.startswith("TF:"):
            curr_res["transfer_learning"] = line.split()[1]
        elif line.startswith("Imgsz:"):
            curr_res["imgsz"] = line.split()[1]
        elif line.startswith("Epochs:"):
            curr_res["epochs"] = line.split()[1]
        elif line.startswith("Batch:"):
            curr_res["batch"] = line.split()[1]
        elif line.startswith("Dataset:"):
            curr_res["dataset"] = line.split()[1]
        elif line.startswith("Loss:"):
            curr_res["optimizer"] = line.split()[1]
        elif line.startswith("Lr0:"):
            try:
                curr_res["lr0"] = float("0."+line.split()[1])
            except:
                curr_res["lr0"] = float(line.split()[1])
            
        if line.strip(" ").startswith("all"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["all_P"] = P
            curr_res["all_R"] = R
            curr_res["all_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["all_mAP@.5"] = mAP5
            curr_res["all_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("knife"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["knife_P"] = P
            curr_res["knife_R"] = R
            curr_res["knife_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["knife_mAP@.5"] = mAP5
            curr_res["knife_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("pistol") or line.strip(" ").startswith("gun"):
            P, R, mAP5, mAP95 = line.split()[3:]
            curr_res["pistol_P"] = P
            curr_res["pistol_R"] = R
            curr_res["pistol_F1"] = (float(P) * float(R)) / (float(P) + float(R))
            curr_res["pistol_mAP@.5"] = mAP5
            curr_res["pistol_mAP@.5:.95"] = mAP95

        elif line.strip(" ").startswith("Results saved to"):
            model_key = line.split("/")[-1].replace('\x1b[0m', '').replace('\n', '')
            model_info = model_key.split("_")

            curr_res["model_key"] = model_key

            results.append(curr_res)
            curr_res = dict()

    df = pd.DataFrame(results)
    df.set_index('model_key', inplace=True)
    return df


file_path = os.path.join(eval_results_dir, output_file)
df = process_file(file_path)
df.to_csv(f'{eval_results_dir}/output_csvs/{output_file_name}_valid.csv')
df

,model,transfer_learning,imgsz,epochs,batch,dataset,optimizer,lr0,all_P,all_R,all_F1,all_mAP@.5,all_mAP@.5:.95
model_key,,,,,,,,,,,,,
model_yolov8_tf_yolov8m_imgsz_640_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8m,640,100,16,v2,SGD,0.010,0.993,0.962,0.488627,0.977,0.941
model_yolov8_tf_yolov8m_imgsz_640_epochs_50_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8m,640,50,16,v2,SGD,0.010,0.991,0.962,0.488142,0.974,0.938
model_yolov8_tf_yolov8s_imgsz_640_epochs_100_batch_16_dataset_v2_loss_Adam_lr_001,yolov8,yolov8s,640,100,16,v2,Adam,0.001,0.986,0.962,0.486926,0.981,0.941
model_yolov8_tf_yolov8s_imgsz_640_epochs_300_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,640,300,16,v2,SGD,0.010,1,0.961,0.490056,0.982,0.942
model_yolov8_tf_yolov8s_imgsz_640_epochs_50_batch_16_dataset_v2_loss_Adam_lr_001,yolov8,yolov8s,640,50,16,v2,Adam,0.001,0.997,0.955,0.487774,0.979,0.932
model_yolov8_tf_yolov8s_imgsz_800_epochs_100_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,800,100,16,v2,SGD,0.010,0.987,0.961,0.486913,0.982,0.949
model_yolov8_tf_yolov8s_imgsz_800_epochs_60_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,800,60,16,v2,SGD,0.010,0.993,0.962,0.488627,0.977,0.946
model_yolov8_tf_yolov8s_imgsz_800_epochs_40_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,800,40,16,v2,SGD,0.010,0.997,0.924,0.479556,0.98,0.932
model_yolov8_tf_yolov8s_imgsz_800_epochs_80_batch_16_dataset_v2_loss_SGD_lr_01,yolov8,yolov8s,800,80,16,v2,SGD,0.010,0.987,0.961,0.486913,0.982,0.948
